In [1]:
from SpinTrain import *
import numpy as np


In [2]:
ca = CommutatorAlgebra()

ca['S+_a', 'S-_a'] = Term([2, 'Sz_a'])
ca['Sz_a', 'S+_a'] = Term(['S+_a'])
ca['Sz_a', 'S-_a'] = Term([-1,'S-_a'])
# by default assumes all objects are scalars

In [18]:
ca = CommutatorAlgebra()

def define_comms(site):
    plus  = 'S^+_{%s}' % site
    minus = 'S^-_{%s}' % site
    z     = 'S^z_{%s}' % site

    ca[plus, minus] = Term([2, z])
    ca[z, plus]  = Term([+1, plus])
    ca[z, minus] = Term([-1, minus])
    
bond_zetas = [r'\zeta_1', r'\zeta_2', r'\zeta_3']
bond_zetabs = [r'\zeta^*_1', r'\zeta^*_2', r'\zeta^*_3']

WA = [1,2,3,4,5,6]
WB = [1,2,3,4,5,6]

WAb = [1,2,3,4,5,6]
WBb = [1,2,3,4,5,6]

# assigns A bonds to (h0h1), (h2h3), (h4h5)

for i in range(6):
    h = 'h%d' % i
    v = 'v%d' % i
    n = 'n%d' % i
    
    hpp = 'h%d' % ((i+1)%6)
    
    hp = 'h%d' % ((i-1)%6)
    vp = 'v%d' % ((i-1)%6)
    np = 'n%d' % ((i-1)%6)
    
    define_comms(h)
    define_comms(v)
    define_comms(n)
    

    
    
    w1 = Expression([bond_zetas[i%3], 'S^z_{%s}' % hpp],
                    [bond_zetas[(i+1)%3], 'S^z_{%s}' % n],
                    [bond_zetas[(i-1)%3], 'S^z_{%s}' % v])
    w2 = Expression([bond_zetas[(i-1)%3], 'S^z_{%s}' % hp],
                    [bond_zetas[(i+1)%3], 'S^z_{%s}' % np],
                    [bond_zetas[i%3], 'S^z_{%s}' % vp])
    w1b = Expression([bond_zetabs[i%3], 'S^z_{%s}' % hpp],
                    [bond_zetabs[(i+1)%3], 'S^z_{%s}' % n],
                    [bond_zetabs[(i-1)%3], 'S^z_{%s}' % v])
    w2b = Expression([bond_zetabs[(i-1)%3], 'S^z_{%s}' % hp],
                    [bond_zetabs[(i+1)%3], 'S^z_{%s}' % np],
                    [bond_zetabs[i%3], 'S^z_{%s}' % vp])
    
    if i %2 == 0:
        WA[i], WB[i] = w1, w2
    else:
        WA[i], WB[i] = w2, w1

def tostar(x):
    res = x.copy()
    for i in range(1,4):
        s = r'\zeta_%d' % i
        sb = r'\zeta^*_%d' % i
        res = res.substitute(s, Expression(sb))
    return res
        
# calculate the complex conjugate
for i in range(6):
    WAb[i] = tostar(WA[i])
    WBb[i] = tostar(WB[i])
        


In [19]:
Z    = [0,0,0,0,0,0]
Zbar = [0,0,0,0,0,0]

for i in range(6):

    Z[i] =  Term(['K_A'])*WA[i]+Term(['K_B'])*WB[i]
    Zbar[i] =  Term(['K_A'])*WAb[i]+Term(['K_B'])*WBb[i]
#     Z[i] =  Term(['K_A'])*WA[h]
#     Zbar[i] =  Term(['K_A'])*WAb[h]
    
for k in range(6):
    print(k)
    Z[k].show()
    Zbar[k].show()

0


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

1


<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

5


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [21]:
# Construct the new commutator algebra
# [Za, Zb = ???]
cb = CommutatorAlgebra()

def simplifyHere(x):
    for i in range(6):
        ca.move_left(x, 'S^z_{n%s}' % i)
        ca.move_left(x, 'S^z_{v%s}' % i)
        ca.move_left(x, 'S^z_{h%s}' % i)

    ca.move_left(x, 'K_A')
    ca.move_left(x, 'K_B')
    ca.move_left(x, '\\zeta_1')
    ca.move_left(x, '\\zeta_2')
    ca.move_left(x, '\\zeta_3')

    x.collect()

for j in range(6):
    for spinstr in ['S^+_{h%d}' % ((j-1)%6), 'S^-_{h%d}' % ((j-1)%6),
                    'S^+_{h%d}' % ((j+1)%6), 'S^-_{h%d}' % ((j+1)%6)]:
#     for spinstr in ['S^+_{h%d}' % ((j)%6), 'S^-_{h%d}' % ((j)%6)]:
    
    
        S = Term([spinstr])

        x1 = S*Z[j] + (-Z[j]*S)
        x2 = S*Zbar[j] + (-Zbar[j]*S)

        ca.move_right(x1, spinstr)
        simplifyHere(x1)
        ca.move_right(x2, spinstr)
        simplifyHere(x2)

        cb[spinstr, 'Z_{h%d}' % j] = x1
        cb[spinstr, 'Z^*_{h%d}' % j] = x2
        
#         print('['+spinstr+', Z'+str(j)+ '] =')
#         x1.show()
#         print('['+spinstr+', Z*'+str(j)+ '] =')
#         x2.show()


In [24]:
cwring = []
ccwring = []

def separateScalars(x):
    ca.move_left(x, '\\zeta_1')
    ca.move_left(x, '\\zeta_2')
    ca.move_left(x, '\\zeta_3')
    ca.move_left(x, '\\zeta^*_1')
    ca.move_left(x, '\\zeta^*_2')
    ca.move_left(x, '\\zeta^*_3')
    ca.move_left(x, 'K_A')
    ca.move_left(x, 'K_B')
    ca.move_left(x, 'U_1')
    ca.move_left(x, 'U_2')
    ca.move_left(x, 'U_3')

def rightyHops(x):
    for l in range(6):
        cb.move_right(x, 'S^+_{h%d}'%l)
        cb.move_right(x, 'S^-_{h%d}'%l)
    
def leftyZ(x):
    for l in range(6):
        cb.move_left(Hring, 'Z_{h%d}'%l)
        cb.move_left(Hring, 'Z^*_{h%d}'%l)
        
    
for i in range(6):


    h0 = 'h%d' % i
    h1 = 'h%d' % ((i+1)%6)
    h2 = 'h%d' % ((i+2)%6)
    h3 = 'h%d' % ((i+3)%6)
    h4 = 'h%d' % ((i+4)%6)
    h5 = 'h%d' % ((i+5)%6)


    F = Expression(['S^-_{%s}' % h0, 'S^+_{%s}' % h1])
    E = Expression(['S^-_{%s}' % h2, 'S^+_{%s}' % h3])
    f = Term('Z^*_{%s}'% h4)*Expression(['S^-_{%s}' % h4])
    e = Term('Z_{%s}'% h5)*Expression(['S^+_{%s}' % h5])

    Hring = Expression()
    
    
    if i%2==0:
        # A-hops
        Hring += Term('U_{1A}')*(f*F*E*e + e*E*F*f) 
        Hring += Term('U_{2A}')*(e*f*E*F + f*e*E*F + E*F*e*f + E*F*f*e)
        Hring += Term('U_{3A}')*(E*f*e*F + F*e*f*E)
        Hring =  Term(['J_{\perp A}','J_{\perp A}'])*Hring
    else:
        Hring += Term('U_{1B}')*(f*F*E*e + e*E*F*f) 
        Hring += Term('U_{2B}')*(e*f*E*F + f*e*E*F + E*F*e*f + E*F*f*e)
        Hring += Term('U_{3B}')*(E*f*e*F + F*e*f*E)
        Hring =  Term(['J_{\perp B}','J_{\perp B}'])*Hring
    
    separateScalars(Hring)    
    rightyHops(Hring)
    leftyZ(Hring)
    separateScalars(Hring)    
    Hring.collect()
    ccwring.append(Hring)
    
    # and the other direction (h.c. nontrivial)
    F = Expression(['S^+_{%s}' % h0, 'S^-_{%s}' % h1])
    E = Expression(['S^+_{%s}' % h2, 'S^-_{%s}' % h3])
    f = Term('Z_{%s}'% h4)*Expression(['S^+_{%s}' % h4])
    e = Term('Z^*_{%s}'% h5)*Expression(['S^-_{%s}' % h5])

    # calculate all the different contributions from the different energy sectors

    Hring = Expression()
    Hring += Term('U_1')*(f*F*E*e + e*E*F*f) 
    Hring += Term('U_2')*(e*f*E*F + f*e*E*F + E*F*e*f + E*F*f*e)
    Hring += Term('U_2')*(E*f*e*F + F*e*f*E)

    separateScalars(Hring)    
    rightyHops(Hring)
    leftyZ(Hring)
    separateScalars(Hring)    
    Hring.collect()
    cwring.append(Hring)

In [42]:
grand = Expression()
for i in range(6):
    grand += cwring[i]+ccwring[i]

grand.collect()
grand.sort()
# grand.show()

O = Term(grand.terms[0].ops[-6:])
r1 = grand.coefficient(O)


In [53]:
ring1 = cwring[0]+cwring[2]+cwring[4]+ccwring[1]+ccwring[3]+ccwring[5]
separateScalars(ring1)
ring1.collect()
ring1.sort()

rpre = ring1.coefficient(O)
rpre.show()
x = r1 + -rpre
x.collect()



<IPython.core.display.Latex object>

True

In [ ]:
l = lambda t : t.ops[0] if len(t.ops)>0 else 0

ls = [Term('Sz'),Term('Sx')]
ls.sort(key=l)
ls

In [ ]:
l = [1,2,3,4,'a']
l[-1:]